In [102]:
from pykrx import stock
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform
import datetime
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

kospi_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
kospi_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

kosdaq_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="KOSDAQ")})
kosdaq_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))


In [124]:
today = datetime.datetime.today()
month_ago = str(today.year) + str(today.month-1) + str(today.day)
month_ago2 = str(today.year) + str(today.month-2) + str(today.day)
today = str(today.year) + str(today.month) + str(today.day)
month_ago

'2021108'

In [125]:
shrink_kospi = stock.get_market_cap_by_ticker(month_ago, market='KOSPI').sort_values('시가총액', ascending=False)
shrink_kospi = shrink_kospi.head(50)
shrink_kosdaq = stock.get_market_cap_by_ticker(month_ago, market='KOSDAQ').sort_values('시가총액', ascending=False)
shrink_kosdaq = shrink_kosdaq.head(50)
name1 = []
name2 = []
for ticker in shrink_kospi.index:
    name1.append(kospi_list[kospi_list['종목코드'] == ticker]['종목명'].values[0])
for ticker in shrink_kosdaq.index:
    name2.append(kosdaq_list[kosdaq_list['종목코드'] == ticker]['종목명'].values[0])
total_name = name1 + name2
len(name1), len(name2), len(total_name)

(50, 50, 100)

In [126]:
def get_name(name):
    name = name
    fromdate = month_ago
    todate = today

    ticker = stock_list.loc[stock_list['종목명']==name, '종목코드']
    df = stock.get_market_ohlcv_by_date(fromdate=fromdate, todate=todate, ticker=ticker)


    df = df.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})
    df['ma20'] = df['Close'].rolling(window=20).mean() # 20일 이동평균
    df['stddev'] = df['Close'].rolling(window=20).std() # 20일 이동표준편차
    df['upper'] = df['ma20'] + 2*df['stddev'] # 상단밴드
    df['lower'] = df['ma20'] - 2*df['stddev'] # 하단밴드
    df = df[19:] # 20일 이동평균을 구했기 때문에 20번째 행부터 값이 들어가 있음
    df['volatility'] = (df['upper'] - df['lower'])/2

    df1 = df.copy().loc[:,('Close','ma20','volatility')]
    df1.insert(0, column='f_close', value=0)
    df1.insert(0, column='f_ma20', value=0)
    df1.insert(0, column='f_vol', value=0)
    df1.insert(0, column='ff_vol', value=0)

    for i in range(1, len(df1)):
        df1['f_vol'][i] = df['volatility'][i] - df['volatility'][i-1]
        df1['f_ma20'][i] = df['ma20'][i] - df['ma20'][i-1]
        df1['f_close'][i] = df['Close'][i] - df['Close'][i-1]
    for i in range(1, len(df1)):
        df1['ff_vol'][i] = df1['f_vol'][i] - df1['f_vol'][i-1]


    scaler = MinMaxScaler()
    df2 = df1.copy()
    df2[:] = scaler.fit_transform(df2[:])
    return df1[::-1], df2[::-1]

In [127]:
select = []
for name in total_name:
    if (get_name(name)[0]['Close'][0] > get_name(name)[0]['ma20'][0]) and (get_name(name)[0]['Close'][1] < get_name(name)[0]['ma20'][1]):
        select.append(name)
select

C:\Users\ghdak\AppData\Local\Temp/ipykernel_20032/224277572.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['f_vol'][i] = df['volatility'][i] - df['volatility'][i-1]
C:\Users\ghdak\AppData\Local\Temp/ipykernel_20032/224277572.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['f_ma20'][i] = df['ma20'][i] - df['ma20'][i-1]
C:\Users\ghdak\AppData\Local\Temp/ipykernel_20032/224277572.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

['대한항공', '미래에셋대우스팩 5호', '아이스크림에듀', '뿌리깊은나무들']